In [1]:
import snowflake.connector
import pandas
import torch
from interactions import Interactions
from cross_validation import random_train_test_split

In [2]:
import snowflake.connector
import pandas as pd

def create_connection():
    ctx = snowflake.connector.connect(
        user='Zan',
        password='Kundalini55!',
        account='livenpay.ap-southeast-2')
    return ctx

def setup_snowflake(cursor):
    cursor.execute("use warehouse zans_wh")
    cursor.execute("use zans_db.postgres")
    
def query(cursor,query):
    cursor.execute(query)
    results = cursor.fetchall()
    return results

timestamped_sql = """
SELECT 
	accountcoupon_couponusehistory.time AS "accountcoupon_couponusehistory.time",
	account_localuser.id  AS "account_localuser.id",
	account_localuser.email  AS "account_localuser.email",
	merchant_merchant.name  AS "merchant_merchant.name",
	case when merchant_merchant.membership_zone_id = 1 then 'Melbourne'
    when merchant_merchant.membership_zone_id = 2 then 'Sydney'
    else null end AS "merchant_merchant.city",
	branch_branch.id  AS "branch_branch.id",
	branch_branch.suburb_id  AS "branch_branch.suburb_id"
FROM account_localuser AS account_localuser
LEFT JOIN accountcoupon_couponusehistory  AS accountcoupon_couponusehistory ON account_localuser.id = accountcoupon_couponusehistory.user_id 
LEFT JOIN branch_branch  AS branch_branch ON accountcoupon_couponusehistory.branch_id = branch_branch.id 
LEFT JOIN merchant_merchant  AS merchant_merchant ON merchant_merchant.id = branch_branch.merchant_id 

WHERE ((CASE WHEN account_localuser.is_staff  THEN 1 ELSE 0 END
) = 0) AND ((CASE
    WHEN DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())) is not null
    AND DATEADD('month', 12, DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE()))) is not null /* date ranges or in the past x days */
    THEN
    CASE
    WHEN accountcoupon_couponusehistory.time >=  TO_TIMESTAMP(DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))
    AND accountcoupon_couponusehistory.time <= TO_TIMESTAMP(DATEADD('month', 12, DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE()))))
    THEN 'This Period'
    WHEN accountcoupon_couponusehistory.time >=
    DATEADD('day',
      DATEDIFF('day',
        TO_TIMESTAMP(DATEADD('month', 12, DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))),
        TO_TIMESTAMP(DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))
      )*-1,
      DATEADD('day', -1,
        TO_TIMESTAMP(DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))
      )
    )
    AND accountcoupon_couponusehistory.time <= TO_TIMESTAMP(DATEADD('day', 1,DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE()))))
    THEN 'Previous Period'
    END
    END
) IS NOT NULL
)
GROUP BY 7,1,2,3,4,5,6
ORDER BY 1 DESC
"""

sql = """
SELECT 
	account_localuser.id  AS "account_localuser.id",
	account_localuser.email  AS "account_localuser.email",
	merchant_merchant.name  AS "merchant_merchant.name",
	case when merchant_merchant.membership_zone_id = 1 then 'Melbourne'
    when merchant_merchant.membership_zone_id = 2 then 'Sydney'
    else null end AS "merchant_merchant.city",
	branch_branch.id  AS "branch_branch.id",
	branch_branch.suburb_id  AS "branch_branch.suburb_id"
FROM account_localuser AS account_localuser
LEFT JOIN accountcoupon_couponusehistory  AS accountcoupon_couponusehistory ON account_localuser.id = accountcoupon_couponusehistory.user_id 
LEFT JOIN branch_branch  AS branch_branch ON accountcoupon_couponusehistory.branch_id = branch_branch.id 
LEFT JOIN merchant_merchant  AS merchant_merchant ON merchant_merchant.id = branch_branch.merchant_id 

WHERE ((CASE WHEN account_localuser.is_staff  THEN 1 ELSE 0 END
) = 0) AND ((CASE
    WHEN DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())) is not null
    AND DATEADD('month', 12, DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE()))) is not null /* date ranges or in the past x days */
    THEN
    CASE
    WHEN accountcoupon_couponusehistory.time >=  TO_TIMESTAMP(DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))
    AND accountcoupon_couponusehistory.time <= TO_TIMESTAMP(DATEADD('month', 12, DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE()))))
    THEN 'This Period'
    WHEN accountcoupon_couponusehistory.time >=
    DATEADD('day',
      DATEDIFF('day',
        TO_TIMESTAMP(DATEADD('month', 12, DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))),
        TO_TIMESTAMP(DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))
      )*-1,
      DATEADD('day', -1,
        TO_TIMESTAMP(DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))
      )
    )
    AND accountcoupon_couponusehistory.time <= TO_TIMESTAMP(DATEADD('day', 1,DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE()))))
    THEN 'Previous Period'
    END
    END
) IS NOT NULL
)
GROUP BY 1,2,3,4,5,6
ORDER BY 1 DESC
"""

ctx = create_connection()
cursor = ctx.cursor()
setup_snowflake(cursor)

TRANSACTIONS = query(cursor,sql)

cursor.close()
ctx.close()

all_data = pd.DataFrame(TRANSACTIONS, columns=['userId','email','merchant','city','branchId','suburb'])
#all_data = pd.DataFrame(TRANSACTIONS, columns=['time','userId','email','merchant','city','branchId','suburb'])

In [3]:
CITY = "Sydney"

cull = ['Uber','Secure Parking Melbourne','Taste Of Melbourne','UberEATS Sydney','Liven Bistro'
       , 'Cardly - You Write, We Post', 'Good Food & Wine Show', 'Sayers Sister [Kounta Demo]'
       , 'Airtasker','Peninsula Hot Springs','Melbourne River Cruises','Grilla House','Margaret River Gourmet Escape','Madeira','Royale Fusion'
        ,'The Bar.Ber','Tandoori Point','The Italian Cucina','Tokyo Bar','The Trust'
        ,"Orita's",'Pelican ','Olive Garden','Golftec','Lash Labs','Cafe on Bourke','Ibuki House','Mint Leaf',"Tina's Noodle Kitchen"
       ,'Uber Sydney','Cardly - You Write, We Post','Secure Parking Sydney','UberEATS Sydney','Liven Bistro','Good Food & Wine Show','Sayers Sister [Kounta Demo]','Airtasker','Taste Of Sydney']

for merchant in cull:
    all_data = all_data[all_data.merchant!=merchant]
    
all_data = all_data.dropna()
all_data.branchId = all_data.branchId.astype(int)
city_data = all_data[all_data.city==CITY]
city_data.head()

,userId,email,merchant,city,branchId,suburb
3,366067,bdm@rnjrealty.com.au,Ajisen Ramen,Sydney,1482,411.0
4,366064,preciousngo@hotmail.com,CoCo Fresh Tea & Juice,Sydney,1246,418.0
13,366045,elizamcgowan@icloud.com,Messina,Sydney,1113,411.0
14,366044,ryan@lonstein.net,BL Burgers,Sydney,1494,422.0
15,366041,jess.lee.phillips@live.com.au,Oh! Matcha,Sydney,1459,411.0


In [63]:
userId,branchId,suburb = city_data['userId'],city_data['branchId'],city_data['suburb']
interactions = pd.DataFrame({"userId":userId,"branchId":branchId,"suburb":suburb})
interactions = interactions[interactions.groupby('userId').userId.transform(len) > 1]

interactions.head()

,userId,branchId,suburb
66,365916,1458,411.0
67,365916,1247,411.0
68,365914,1115,489.0
69,365914,1575,411.0
104,365427,1538,421.0


In [64]:
from collections import defaultdict

userIds = interactions.userId.unique()
itemIds = interactions.branchId.unique()
internal_userIds = defaultdict(lambda: len(internal_userIds))
internal_itemIds = defaultdict(lambda: len(internal_itemIds))

external_userIds = {}
external_itemIds = {}

for user,item in zip(interactions.userId.values,interactions.branchId.values):
    internal_userIds[user]
    internal_itemIds[item]
    external_userIds[internal_userIds[user]] = user
    external_itemIds[internal_itemIds[item]] = item

In [65]:
interactions.head()

,userId,branchId,suburb
66,365916,1458,411.0
67,365916,1247,411.0
68,365914,1115,489.0
69,365914,1575,411.0
104,365427,1538,421.0


In [66]:
interactions_proc = interactions

internalItems = []
internalUsers = []

for i,r in interactions.iterrows():
    internalItems += [internal_itemIds[int(r.branchId)]]
    internalUsers += [internal_userIds[int(r.userId)]]
    
interactions_proc['branchId'] = internalItems
interactions_proc['userId'] = internalUsers

In [67]:
interactions_proc.loc[interactions_proc['userId']==1]

,userId,branchId,suburb
68,1,2,489.0
69,1,3,411.0


In [68]:
userIds = interactions_proc.userId.values
itemIds = interactions_proc.branchId.values
suburbs = interactions_proc.suburb.values

In [113]:
interactions_proc.head()

,userId,branchId,suburb
66,0,0,411.0
67,0,1,411.0
68,1,2,489.0
69,1,3,411.0
104,2,4,421.0


In [123]:
featureDict = {}

for i in itemIds:
    featureDict[i] = int(interactions_proc.loc[interactions_proc['branchId']==i]['suburb'].unique())

In [127]:
item_features = list(featureDict.values())

In [129]:
type(userIds)

numpy.ndarray

In [130]:
feature_interactions = Interactions(userIds,itemIds,item_features=np.asarray(item_features))

In [131]:
feature_interactions

<Interactions dataset (13496 users x 314 items x 50926 interactions)>

In [132]:
from scipy import sparse

user_identity = sparse.identity(no_features.num_users)
item_identity = sparse.identity(no_features.num_items)

In [133]:
feature_interactions.item_features

array([  411.,   411.,   489.,   411.,   421.,   411.,   411.,   418.,
         411.,   411.,   411.,   408.,   408.,   408.,   411.,   411.,
         408.,   418.,   408.,   408.,   411.,   411.,   408.,   426.,
         411.,   547.,   411.,   701.,   411.,   677.,   411.,   411.,
         422.,   411.,   411.,   408.,   411.,   422.,   411.,   411.,
         411.,   411.,   411.,   411.,   453.,   411.,   411.,   534.,
         526.,   547.,   411.,   547.,   678.,   718.,   419.,   419.,
         521.,   408., 16082.,   423.,   499.,   772.,   411.,   419.,
         408.,   547.,   772.,   419.,   419.,   418.,   521.,   411.,
         419.,   470.,   474.,   419.,   499.,   411.,   411.,   434.,
         408.,   411.,   711.,   411.,   408., 16082.,   411.,   411.,
         411.,   892.,   411.,   411.,   435.,   411.,   436.,   417.,
         411.,   411.,   423.,   411.,   470.,   411.,   446.,   411.,
         411.,   411.,   958.,   836.,   411.,   892.,   419.,   673.,
      

In [134]:
"""
Factorization models for implicit feedback problems.
"""

import numpy as np

import torch

import torch.optim as optim

from torch.autograd import Variable

from helpers import _repr_model
from factorization._components import (_predict_process_features,
                                                 _predict_process_ids)
from losses import (adaptive_hinge_loss,
                              bpr_loss,
                              hinge_loss,
                              pointwise_loss)

from factorization.representations import (BilinearNet,
                                                     FeatureNet,
                                                     HybridContainer)

from sampling import sample_items
from torch_utils import cpu, gpu, set_seed


class ImplicitFactorizationModel(object):
    """
    An implict feedback matrix factorization model. Uses a classic
    matrix factorization [1]_ approach, with latent vectors used
    to represent both users and items. Their dot product gives the
    predicted score for a user-item pair.

    The latent representation is given by
    :class:`spotlight.factorization.representations.BilinearNet`.

    The model is trained through negative sampling: for any known
    user-item pair, one or more items are randomly sampled to act
    as negatives (expressing a lack of preference by the user for
    the sampled item).

    .. [1] Koren, Yehuda, Robert Bell, and Chris Volinsky.
       "Matrix factorization techniques for recommender systems."
       Computer 42.8 (2009).

    Parameters
    ----------

    loss: string, optional
        One of 'pointwise', 'bpr', 'hinge', or 'adaptive hinge',
        corresponding to losses from :class:`spotlight.losses`.
    embedding_dim: int, optional
        Number of embedding dimensions to use for users and items.
    n_iter: int, optional
        Number of iterations to run.
    batch_size: int, optional
        Minibatch size.
    l2: float, optional
        L2 loss penalty.
    learning_rate: float, optional
        Initial learning rate.
    optimizer_func: function, optional
        Function that takes in module parameters as the first argument and
        returns an instance of a Pytorch optimizer. Overrides l2 and learning
        rate if supplied. If no optimizer supplied, then use ADAM by default.
    use_cuda: boolean, optional
        Run the model on a GPU.
    sparse: boolean, optional
        Use sparse gradients for embedding layers.
    random_state: instance of numpy.random.RandomState, optional
        Random state to use when fitting.
    """

    def __init__(self,
                 loss='pointwise',
                 embedding_dim=32,
                 n_iter=10,
                 batch_size=256,
                 l2=0.0,
                 learning_rate=1e-2,
                 optimizer_func=None,
                 use_cuda=False,
                 sparse=False,
                 random_state=None):

        assert loss in ('pointwise',
                        'bpr',
                        'hinge',
                        'adaptive_hinge')

        self._loss = loss
        self._embedding_dim = embedding_dim
        self._n_iter = n_iter
        self._learning_rate = learning_rate
        self._batch_size = batch_size
        self._l2 = l2
        self._use_cuda = use_cuda
        self._sparse = sparse
        self._optimizer_func = optimizer_func
        self._random_state = random_state or np.random.RandomState()

        self._num_users = None
        self._num_items = None
        self._net = None
        self._optimizer = None
        self._loss_func = None

        set_seed(self._random_state.randint(-10**8, 10**8),
                 cuda=self._use_cuda)

    def __repr__(self):

        return _repr_model(self)

    @property
    def _initialized(self):
        return self._net is not None

    def _initialize(self, interactions):

        (self._num_users,
         self._num_items) = (interactions.num_users,
                             interactions.num_items)

        latent_net = BilinearNet(self._num_users,
                                 self._num_items,
                                 self._embedding_dim,
                                 sparse=self._sparse)

        if interactions.num_user_features():
            user_net = FeatureNet(interactions.num_user_features(),
                                  self._embedding_dim)
        else:
            user_net = None

        if interactions.num_context_features():
            context_net = FeatureNet(interactions.num_context_features(),
                                     self._embedding_dim)
        else:
            context_net = None

        if interactions.num_item_features():
            item_net = FeatureNet(interactions.num_item_features(),
                                  self._embedding_dim)
        else:
            item_net = None

        self._net = gpu(HybridContainer(latent_net,
                                        user_net,
                                        context_net,
                                        item_net),
                        self._use_cuda)

        if self._optimizer_func is None:
            self._optimizer = optim.Adam(
                self._net.parameters(),
                weight_decay=self._l2,
                lr=self._learning_rate
            )
        else:
            self._optimizer = self._optimizer_func(self._net.parameters())

        if self._loss == 'pointwise':
            self._loss_func = pointwise_loss
        elif self._loss == 'bpr':
            self._loss_func = bpr_loss
        elif self._loss == 'hinge':
            self._loss_func = hinge_loss
        else:
            self._loss_func = adaptive_hinge_loss

    def _check_input(self, user_ids, item_ids, allow_items_none=False):

        if isinstance(user_ids, int):
            user_id_max = user_ids
        else:
            user_id_max = user_ids.max()

        if user_id_max >= self._num_users:
            raise ValueError('Maximum user id greater '
                             'than number of users in model.')

        if allow_items_none and item_ids is None:
            return

        if isinstance(item_ids, int):
            item_id_max = item_ids
        else:
            item_id_max = item_ids.max()

        if item_id_max >= self._num_items:
            raise ValueError('Maximum item id greater '
                             'than number of items in model.')

    def fit(self, interactions, verbose=False):
        """
        Fit the model.

        When called repeatedly, model fitting will resume from
        the point at which training stopped in the previous fit
        call.

        Parameters
        ----------

        interactions: :class:`spotlight.interactions.Interactions`
            The input dataset.
        """

        user_ids = interactions.user_ids.astype(np.int64)
        item_ids = interactions.item_ids.astype(np.int64)

        if not self._initialized:
            self._initialize(interactions)

        self._check_input(user_ids, item_ids)

        for epoch_num in range(self._n_iter):

            interactions.shuffle(random_state=self._random_state)

            epoch_loss = 0.0

            for (minibatch_num,
                 minibatch) in enumerate(interactions
                                         .minibatches(use_cuda=self._use_cuda,
                                                      batch_size=self._batch_size)):

                minibatch = minibatch.torch(self._use_cuda).variable()

                positive_prediction = self._net(minibatch.user_ids,
                                                minibatch.item_ids,
                                                minibatch.user_features,
                                                minibatch.context_features,
                                                minibatch.get_item_features(
                                                    minibatch.item_ids
                                                ))
                
                if self._loss == 'adaptive_hinge':
                    negative_prediction = [self._get_negative_prediction(minibatch)
                                           for _ in range(5)]
                else:
                    negative_prediction = self._get_negative_prediction(minibatch)

                self._optimizer.zero_grad()

                loss = self._loss_func(
                    positive_prediction,
                    negative_prediction,
                    weights=minibatch.weights
                )

                epoch_loss += loss.data[0]

                loss.backward()
                self._optimizer.step()

            epoch_loss /= minibatch_num + 1

            if verbose:
                print('Epoch {}: loss {}'.format(epoch_num, epoch_loss))

    def _get_negative_prediction(self, minibatch):

        negative_items = sample_items(
            self._num_items,
            len(minibatch),
            random_state=self._random_state)
        negative_var = Variable(
            gpu(torch.from_numpy(negative_items), self._use_cuda)
        )

        negative_prediction = self._net(minibatch.user_ids,
                                        negative_var,
                                        minibatch.user_features,
                                        minibatch.context_features,
                                        minibatch.get_item_features(
                                            negative_var
                                        ))

        return negative_prediction

    def predict(self, user_ids, item_ids=None,
                user_features=None,
                context_features=None,
                item_features=None):
        """
        Make predictions: given a user id, compute the recommendation
        scores for items.

        Parameters
        ----------

        user_ids: int or array
           If int, will predict the recommendation scores for this
           user for all items in item_ids. If an array, will predict
           scores for all (user, item) pairs defined by user_ids and
           item_ids.
        item_ids: array, optional
            Array containing the item ids for which prediction scores
            are desired. If not supplied, predictions for all items
            will be computed.

        Returns
        -------

        predictions: np.array
            Predicted scores for all items in item_ids.
        """

        self._check_input(user_ids, item_ids, allow_items_none=True)
        self._net.train(False)

        user_ids, item_ids = _predict_process_ids(user_ids, item_ids,
                                                  self._num_items,
                                                  self._use_cuda)

        (user_features,
         context_features,
         item_features) = _predict_process_features(user_features,
                                                    context_features,
                                                    item_features,
                                                    len(item_ids),
                                                    self._use_cuda)

        out = self._net(user_ids,
                        item_ids,
                        user_features,
                        context_features,
                        item_features)

        return cpu(out.data).numpy().flatten()

In [169]:
train,test = random_train_test_split(feature_interactions)

model = ImplicitFactorizationModel(n_iter=10,
                                   loss='adaptive_hinge',
                                        l2=-1.252542352e-10)
model.fit(train,verbose=True)


input dim 314
output dim 32


/opt/conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:253: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch 0: loss 0.8619645833969116
Epoch 1: loss 0.6869610548019409
Epoch 2: loss 0.4557695984840393
Epoch 3: loss 0.3175075948238373
Epoch 4: loss 0.2591142952442169
Epoch 5: loss 0.2299012839794159
Epoch 6: loss 0.21158790588378906
Epoch 7: loss 0.19884344935417175
Epoch 8: loss 0.18977171182632446
Epoch 9: loss 0.18617965281009674


In [170]:
from evaluation import mrr_score
mrr = mrr_score(model, test)
print('mean mrr',mrr.mean())

mean mrr 0.11111769061025172


In [144]:
model

<ImplicitFactorizationModel: HybridContainer(
  (latent): BilinearNet(
    (user_embeddings): ScaledEmbedding(13496, 32)
    (item_embeddings): ScaledEmbedding(314, 32)
    (user_biases): ZeroEmbedding(13496, 1)
    (item_biases): ZeroEmbedding(314, 1)
  )
  (item): FeatureNet(
    (fc_1): Linear(in_features=314, out_features=32, bias=False)
  )
)>

In [145]:
for i in model._net.parameters():
    print(i)

Parameter containing:
tensor([[-6.5755e-02, -3.2030e-01, -2.9230e-01,  ...,  2.6644e-01,
          1.4790e-01,  1.1464e-01],
        [-1.8127e-01,  1.9440e-01,  5.3314e-01,  ..., -5.3931e-01,
          3.7139e-01, -2.8627e-01],
        [ 2.4109e-01, -1.0177e-01, -1.2552e-01,  ..., -3.8124e-03,
          4.3070e-01, -5.3761e-01],
        ...,
        [-6.2359e-02,  2.9966e-01,  3.8733e-01,  ...,  2.4788e-02,
          2.5672e-01, -8.5203e-04],
        [-1.4856e-01, -1.1168e-01,  3.3642e-01,  ...,  1.8984e-01,
          5.3959e-01, -4.7349e-01],
        [-4.3191e-01, -6.8883e-01, -3.0146e-01,  ..., -3.0254e-01,
          9.2342e-02, -4.9786e-01]])
Parameter containing:
tensor([[-5.5381e-02,  4.3892e-03, -1.1880e+00,  ...,  1.5066e-01,
          6.9926e-01,  1.4968e-01],
        [ 9.4241e-01, -2.5111e-01, -5.3273e-01,  ...,  6.0783e-01,
          5.2881e-01,  1.1163e-01],
        [-7.0575e-01, -1.0381e+00,  4.3856e-01,  ..., -1.1016e-01,
          7.7070e-01, -5.4566e-01],
        ...,
  

In [156]:
learned = list(model._net.parameters())

user_embeddings = learned[0]
item_embeddings = learned[1]

user_embeddings = list(zip(user_embeddings,range(0,feature_interactions.num_users)))
item_embeddings = list(zip(item_embeddings,range(0,feature_interactions.num_items)))


In [157]:
external_userIds[13495]

24

In [158]:
no_features.num_users

13496

In [159]:
user_dict = {}

for emb,u in user_embeddings:
    try:
        user_dict[external_userIds[u]] = emb
    except:
        print(u)

user_dict

{365916: tensor([-0.0658, -0.3203, -0.2923, -0.0110, -0.1446, -0.0870,  0.4366,
         -0.0330, -0.2088,  0.1635,  0.1512,  0.0919,  0.4360, -0.1019,
          0.2614, -0.0203,  0.0508,  0.5167, -0.2410,  0.0427,  0.1221,
          0.5917, -0.5371,  0.0901, -0.0106, -0.2965, -0.2824, -0.0645,
         -0.0174,  0.2664,  0.1479,  0.1146]),
 365914: tensor([-0.1813,  0.1944,  0.5331, -0.2293, -0.2350,  0.0196,  0.1834,
         -0.3703, -0.6397, -0.0606,  0.6722, -0.6694,  0.1991, -0.2442,
         -0.6649,  0.7006, -1.0644,  0.2760,  0.3710,  0.2819, -0.0367,
          0.3390,  0.2143,  0.3966,  0.1012, -0.1558, -0.5022,  0.5151,
         -0.3062, -0.5393,  0.3714, -0.2863]),
 365427: tensor([ 0.2411, -0.1018, -0.1255, -0.2853,  0.2918, -0.0234,  0.3673,
          0.3034,  0.2452, -0.5303, -0.1229,  0.5410, -0.0384, -0.0878,
         -0.4004,  0.1688,  0.0739,  0.1257, -0.1440,  0.0436, -0.2702,
          0.2458,  0.2646,  0.2546, -0.2243,  0.1500,  0.4419,  0.0860,
          0.0471, 

In [161]:
item_dict = {}

for emb,i in item_embeddings:
    try:
        item_dict[external_itemIds[i]] = emb
    except:
        print(u)

item_dict

{1458: tensor([-0.0554,  0.0044, -1.1880,  0.1841, -0.1091, -0.3677, -0.1180,
         -0.2523,  0.4430, -0.5150, -0.3904,  0.7502,  0.8160, -1.1818,
          1.2369, -0.5844,  0.0546,  1.1051, -0.8523,  0.4272, -0.4728,
          0.8959, -0.7084,  0.1338, -0.6352, -0.5859, -0.1849, -0.1290,
          0.4730,  0.1507,  0.6993,  0.1497]),
 1247: tensor([ 0.9424, -0.2511, -0.5327,  0.3759, -0.0095, -0.6226,  0.2052,
          0.3076, -0.6960, -0.1853, -0.1527,  0.9723,  0.5249,  0.4550,
         -0.0253,  0.7035,  0.4035,  0.2442, -0.1926,  0.1295, -0.4499,
          0.6704,  0.0605, -0.2180, -0.1731, -0.9019,  0.1490,  0.1451,
         -0.2432,  0.6078,  0.5288,  0.1116]),
 1115: tensor([-0.7057, -1.0381,  0.4386, -1.0542, -0.1706, -0.2500, -1.2531,
         -0.6366, -0.7888,  0.1673,  0.6715, -1.0969, -0.0044, -0.5747,
         -0.5846,  0.3771, -1.2075,  0.2772,  0.4014,  0.1957,  0.2567,
          0.6456,  0.3303,  1.3753, -0.2363, -0.4280, -0.6372, -0.6034,
         -0.2622, -0.110

In [165]:
fnc = lambda x:x[0]

fnc(item_embeddings)

(tensor([-0.0554,  0.0044, -1.1880,  0.1841, -0.1091, -0.3677, -0.1180,
         -0.2523,  0.4430, -0.5150, -0.3904,  0.7502,  0.8160, -1.1818,
          1.2369, -0.5844,  0.0546,  1.1051, -0.8523,  0.4272, -0.4728,
          0.8959, -0.7084,  0.1338, -0.6352, -0.5859, -0.1849, -0.1290,
          0.4730,  0.1507,  0.6993,  0.1497]), 0)

In [175]:
userId=317318

user = list(zip(range(feature_interactions.num_items),[user_dict[userId].dot(emb) for emb in learned[1]]))

In [178]:
preds = sorted(user317318,key=lambda x:x[1],reverse=True)

In [188]:
city_data[city_data['userId']==userId]

,userId,email,merchant,city,branchId,suburb
23047,317318,insight_1912@hotmail.com,YiFang Taiwan Fruit Tea,Sydney,1551,701.0
23048,317318,insight_1912@hotmail.com,Sexy Tea,Sydney,1468,716.0
23049,317318,insight_1912@hotmail.com,HongKong Kitchen,Sydney,1508,411.0


In [190]:
for i in range(10):
    branchId = external_itemIds[preds[i][0]]
    x = city_data[city_data['branchId']==branchId]
    merchant = x.merchant.unique()
    suburb = x.suburb.unique()
    print(preds[i][1],' ------ ',merchant[0],': ',str(suburb[0]))
    

tensor(5.2356)  ------  YiFang Taiwan Fruit Tea :  701.0
tensor(3.5313)  ------  N2 Extreme Gelato :  411.0
tensor(3.4333)  ------  HongKong Kitchen :  411.0
tensor(3.3407)  ------  The Caffeine Project :  419.0
tensor(3.2237)  ------  Mad Mex :  436.0
tensor(2.9801)  ------  Roll'd :  772.0
tensor(2.8268)  ------  Two Sticks :  408.0
tensor(2.7856)  ------  Sexy Tea :  716.0
tensor(2.7418)  ------  Roll'd :  411.0
tensor(2.6854)  ------  Meet Fresh :  408.0
